In [1]:
# https://data.bts.gov/Research-and-Statistics/Trips-by-Distance/w96p-f2qv
# mobility
#     1. Extract data from https://data.bts.gov/Research-and-Statistics/Trips-by-Distance/w96p-f2qv using API
#     2. Create a Dataframe with columns we will need alone - 
#         Date,
#         State Postal Code,
#         County Name,
#         Population Staying at Home,
#         Population Not Staying at Home
#     2. Extract data from https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
#     3. Create a dataframe with columns we will need alone - 
#         Date,
#         State,
#         County,
#         Cases,
#         Deaths
#     4. Merge the two Dataframes using Date, State and County names to have the following columns in resulting Dataframe
#         Date,
#         State,
#         County,
#         Population Staying at Home,
#         Population Not Staying at Home,
#         Cases,
#         Deaths
#     5. Scatter Plot and check correlation between covid cases and population staying at home
#     6. Map with the data

In [104]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import numpy as np
from sodapy import Socrata
from config import my_app_token, key_id, key_secret

In [2]:
# Mobility data retrieval
client = Socrata("data.bts.gov",
                 my_app_token,
                 username=key_id,
                 password=key_secret)

# Results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("w96p-f2qv", limit=2028190)

# Convert to pandas DataFrame
mobility_df = pd.DataFrame.from_records(results)
mobility_df.head(5)

,level,date,state_fips,state_code,county_fips,county,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,County,2020-10-03T00:00:00.000,01,AL,01001,Autauga County,10920.0,44681.0,159472.0,32341.0,38888.0,25231.0,25128.0,26034.0,7052.0,3108.0,1497.0,172.0,21.0
1,County,2020-10-02T00:00:00.000,01,AL,01001,Autauga County,9857.0,45744.0,179089.0,40822.0,43890.0,21287.0,27059.0,34514.0,7472.0,2505.0,1223.0,295.0,22.0
2,County,2020-10-01T00:00:00.000,01,AL,01001,Autauga County,9073.0,46528.0,169030.0,33225.0,40020.0,24366.0,23596.0,34141.0,8671.0,3614.0,1044.0,339.0,14.0
3,County,2020-09-30T00:00:00.000,01,AL,01001,Autauga County,11044.0,44557.0,163850.0,30293.0,40399.0,21267.0,23737.0,34307.0,9107.0,3409.0,1265.0,51.0,15.0
4,County,2020-09-29T00:00:00.000,01,AL,01001,Autauga County,9830.0,45771.0,159342.0,32922.0,34968.0,21025.0,22947.0,34139.0,8155.0,4053.0,1029.0,92.0,12.0


In [3]:
mobility_df = mobility_df.loc[mobility_df["level"]=="County"]
mobility_df

,level,date,state_fips,state_code,county_fips,county,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,County,2020-10-03T00:00:00.000,01,AL,01001,Autauga County,10920.0,44681.0,159472.0,32341.0,38888.0,25231.0,25128.0,26034.0,7052.0,3108.0,1497.0,172.0,21.0
1,County,2020-10-02T00:00:00.000,01,AL,01001,Autauga County,9857.0,45744.0,179089.0,40822.0,43890.0,21287.0,27059.0,34514.0,7472.0,2505.0,1223.0,295.0,22.0
2,County,2020-10-01T00:00:00.000,01,AL,01001,Autauga County,9073.0,46528.0,169030.0,33225.0,40020.0,24366.0,23596.0,34141.0,8671.0,3614.0,1044.0,339.0,14.0
3,County,2020-09-30T00:00:00.000,01,AL,01001,Autauga County,11044.0,44557.0,163850.0,30293.0,40399.0,21267.0,23737.0,34307.0,9107.0,3409.0,1265.0,51.0,15.0
4,County,2020-09-29T00:00:00.000,01,AL,01001,Autauga County,9830.0,45771.0,159342.0,32922.0,34968.0,21025.0,22947.0,34139.0,8155.0,4053.0,1029.0,92.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017159,County,2019-01-05T00:00:00.000,56,WY,56045,Weston County,1358.0,5588.0,25218.0,3816.0,4338.0,1546.0,513.0,2832.0,1930.0,9336.0,577.0,235.0,95.0
2017160,County,2019-01-04T00:00:00.000,56,WY,56045,Weston County,1375.0,5571.0,27918.0,4681.0,4405.0,1279.0,672.0,3017.0,2127.0,11052.0,415.0,168.0,102.0
2017161,County,2019-01-03T00:00:00.000,56,WY,56045,Weston County,1346.0,5600.0,20958.0,3719.0,3946.0,1014.0,659.0,1664.0,1981.0,7080.0,596.0,212.0,87.0
2017162,County,2019-01-02T00:00:00.000,56,WY,56045,Weston County,1385.0,5561.0,21634.0,3110.0,4474.0,1380.0,619.0,1956.0,1469.0,7782.0,692.0,86.0,66.0


In [4]:
# Set the county-level covid data file path
covid_data_url = os.path.join("Resources", "us-counties.csv")

# Read the county-level covid data from csv file into DataFrame
covid_df = pd.read_csv(covid_data_url)
covid_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [5]:
# Remove NaNs from the Dataset
print(len(covid_df))
covid_df = covid_df.dropna(how="any")
print(len(covid_df))

615479
609559


In [6]:
# Convert the fips from float to int
covid_df["fips"] = covid_df["fips"].astype(int)
covid_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
615474,2020-10-09,Sweetwater,Wyoming,56037,393,2
615475,2020-10-09,Teton,Wyoming,56039,656,1
615476,2020-10-09,Uinta,Wyoming,56041,378,2
615477,2020-10-09,Washakie,Wyoming,56043,122,6


In [40]:
# Convert "date" column to "datetime64" datatype
mobility_df['date'] = pd.to_datetime(mobility_df['date']) 
mobility_df.dtypes

level                           object
date                    datetime64[ns]
state_fips                      object
state_code                      object
county_fips                      int32
county                          object
pop_stay_at_home                object
pop_not_stay_at_home            object
trips                           object
trips_1                         object
trips_1_3                       object
trips_3_5                       object
trips_5_10                      object
trips_10_25                     object
trips_25_50                     object
trips_50_100                    object
trips_100_250                   object
trips_250_500                   object
trips_500                       object
dtype: object

In [41]:
# Convert "date" column to "datetime64" datatype
covid_df['date'] = pd.to_datetime(covid_df['date']) 
covid_df.dtypes

date      datetime64[ns]
county            object
state             object
fips               int32
cases              int64
deaths             int64
dtype: object

In [42]:
# Convert county_fips to "int" datatype
mobility_dt_df["county_fips"] = mobility_dt_df["county_fips"].astype(int)

In [114]:
# Merge the dataframes
merged_df = pd.merge(mobility_dt_df, covid_df, left_on=["date","county_fips"], right_on=["date","fips"], how="inner")
merged_df

,level,date,state_fips,state_code,county_fips,county_x,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,...,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,county_y,state,fips,cases,deaths
0,County,2020-10-03,01,AL,1001,Autauga County,10920.0,44681.0,159472.0,32341.0,...,7052.0,3108.0,1497.0,172.0,21.0,Autauga,Alabama,1001,1818,27
1,County,2020-10-02,01,AL,1001,Autauga County,9857.0,45744.0,179089.0,40822.0,...,7472.0,2505.0,1223.0,295.0,22.0,Autauga,Alabama,1001,1805,27
2,County,2020-10-01,01,AL,1001,Autauga County,9073.0,46528.0,169030.0,33225.0,...,8671.0,3614.0,1044.0,339.0,14.0,Autauga,Alabama,1001,1798,28
3,County,2020-09-30,01,AL,1001,Autauga County,11044.0,44557.0,163850.0,30293.0,...,9107.0,3409.0,1265.0,51.0,15.0,Autauga,Alabama,1001,1791,27
4,County,2020-09-29,01,AL,1001,Autauga County,9830.0,45771.0,159342.0,32922.0,...,8155.0,4053.0,1029.0,92.0,12.0,Autauga,Alabama,1001,1787,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577757,County,2020-05-31,56,WY,56045,Weston County,1433.0,5534.0,17422.0,2507.0,...,2361.0,5771.0,402.0,66.0,32.0,Weston,Wyoming,56045,1,0
577758,County,2020-05-30,56,WY,56045,Weston County,1348.0,5619.0,19863.0,4508.0,...,1870.0,6153.0,171.0,24.0,29.0,Weston,Wyoming,56045,1,0
577759,County,2020-05-29,56,WY,56045,Weston County,1475.0,5492.0,25077.0,6074.0,...,1694.0,8431.0,484.0,67.0,9.0,Weston,Wyoming,56045,1,0
577760,County,2020-05-28,56,WY,56045,Weston County,1586.0,5381.0,25754.0,6074.0,...,3027.0,5733.0,370.0,51.0,11.0,Weston,Wyoming,56045,1,0


In [115]:
# Include only the columns required
merged_df = merged_df[["date",
                       "state",
                       "county_y",
                       "county_fips",
                       "pop_stay_at_home",
                       "pop_not_stay_at_home", 
                       "cases",
                       "deaths"]]
merged_df

,date,state,county_y,county_fips,pop_stay_at_home,pop_not_stay_at_home,cases,deaths
0,2020-10-03,Alabama,Autauga,1001,10920.0,44681.0,1818,27
1,2020-10-02,Alabama,Autauga,1001,9857.0,45744.0,1805,27
2,2020-10-01,Alabama,Autauga,1001,9073.0,46528.0,1798,28
3,2020-09-30,Alabama,Autauga,1001,11044.0,44557.0,1791,27
4,2020-09-29,Alabama,Autauga,1001,9830.0,45771.0,1787,27
...,...,...,...,...,...,...,...,...
577757,2020-05-31,Wyoming,Weston,56045,1433.0,5534.0,1,0
577758,2020-05-30,Wyoming,Weston,56045,1348.0,5619.0,1,0
577759,2020-05-29,Wyoming,Weston,56045,1475.0,5492.0,1,0
577760,2020-05-28,Wyoming,Weston,56045,1586.0,5381.0,1,0


In [116]:
# Remove NaNs from the Dataset
print(len(merged_df))
merged_df = merged_df.dropna(how="any")
print(len(merged_df))

577762
576711
